In [1]:
from BlackBox.lmart.lMart_utils import LMARTGridsearch
import pandas as pd
%reload_ext autoreload
%autoreload 2

# Grid-search

In [2]:
lMart_parameter = dict(
    boosting_type=["dart", "gbdt"],
    num_leaves=[2, 5, 10, 20, 30, 40],
    max_depth=[-1],
    n_estimators=[75, 80, 100, 150, 200],
    learning_rate=[0.02, 0.05, 0.08, 0.1, 0.15, 0.2],
    reg_lambda=[0.00005, 0.0001, 0.0002, 0.0003]
)

gs = LMARTGridsearch(path_dataset="../../outputs/scores.csv", random_state=841)
best_ = gs.grid_search(lMart_parameter)
gs.save_model_parameters(best_[0],best_[1])

100%|██████████| 1440/1440 [14:58<00:00,  1.60it/s, nDCG_15=0.79]


# Metric evaluation

In [8]:
best_model, parameters = gs.load_model_parameters()
best_model,parameters

(LGBMRanker(boosting_type='dart', class_weight='balanced', force_row_wise=True,
            importance_type='gain', learning_rate=0.15, metric='ndcg',
            n_estimators=75, num_leaves=5, objective='lambdarank',
            reg_lambda=0.0002, verbose=-1),
 {'boosting_type': 'dart',
  'learning_rate': 0.15,
  'max_depth': -1,
  'n_estimators': 75,
  'num_leaves': 5,
  'reg_lambda': 0.0002})

In [9]:
# nDCG
nDCG_train = gs.eval_model(model=best_model, df=gs.train, qIds=gs.qIds_train, nDCG_at=[1,10,15])
nDCG_valid = gs.eval_model(model=best_model, df=gs.valid, qIds=gs.qIds_val, nDCG_at=[1,10,15])
nDCG_test = gs.eval_model(model=best_model, df=gs.test, qIds=gs.qIds_test, nDCG_at=[1,10,15])

display(pd.DataFrame([nDCG_train,nDCG_valid,nDCG_test],index=["Training","Validation","Test"]))

,nDCG@1,nDCG@10,nDCG@15
Training,0.8330,0.8234,0.8144
Validation,0.8132,0.7997,0.7898
Test,0.8069,0.8096,0.7972


# Example of Job-offer

In [10]:
qId= 1
job_curricula = gs.test[gs.test["qId"] == qId]

y_pred = best_model.predict(job_curricula.iloc[:,2:13])

y_pred = pd.DataFrame(y_pred, index=job_curricula.index, columns=["lambdas"])
dt_final = pd.merge(job_curricula, y_pred, left_index=True, right_index=True)

In [11]:
dt_final.sort_values("labels",ascending=False)["labels"].head(15)

1496    4
791     4
1010    3
1190    3
1418    3
1481    3
877     3
883     3
1269    3
1244    3
926     3
1224    3
928     3
1178    3
1326    3
Name: labels, dtype: int64

In [12]:
dt_final.sort_values("lambdas",ascending=False)["labels"].head(15)

1190    3
791     4
1475    2
1398    3
1430    2
1070    3
759     3
935     2
1416    2
1335    2
1244    3
945     3
1401    3
1418    3
999     3
Name: labels, dtype: int64